In [9]:
import glob
import ROOT
from ROOT import TH1D, TH2D, TTree,TFile, TUnfoldSys, TUnfoldDensity, TUnfold, TDirectoryFile
from random import gauss
import numpy as np
import root_numpy as rtnpy
from root_numpy import *
import h5py
import matplotlib.pyplot as plt
import seaborn as sns


### Get list of nanoskim files to run over and sum gen event weights (ie, n_events) for a given samples

In [10]:
%jsroot on

In [11]:
def getListOfFiles(dirName, histos=False):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    #To ensure event counts don't come up against  a histogram file and rather just the nanoskim
    if histos==False: ftype="nanoskim"
    else: ftype="histogram"
    listOfFile[:] = [x for x in listOfFile if "%s"%ftype in x]
    #print (listOfFile)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):            
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

#print (getListOfFiles('/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/ZZ_TuneCP5_13TeV-pythia8/jetObservables_Skimmer_UL17_ZZ_v1/201125_043451/0000/'))

In [12]:
import argparse, os, shutil, sys
import ROOT
from array import array
from collections import OrderedDict
sys.path.insert(0,'../test/')
from Samples_WtopSel import checkDict, dictSamples

sys.path.insert(0,'../python/')
import CMS_lumi as CMS_lumi
import tdrstyle as tdrstyle




####gReset()
ROOT.gROOT.SetBatch()
ROOT.gROOT.ForceStyle()
tdrstyle.setTDRStyle()
ROOT.gStyle.SetOptStat(0)

colorPallete = [ 0, 2, 4, 8, 12, 28, 30 ]


def loadHistograms( samples, var, sel, sysUnc=[], isMC=True, addGenInfo=True, respOnly=False, lumi=1., variables={}, year='2017', process='MC' ):
    """docstring for loadHistograms"""

    SYSUNC = [ '_nom' ] + [ s+u for u in ['Up', 'Down'] for s in sysUnc ]

    allHistos = {}
    for isam in samples:
        tmpList = [ 'reco'+var+syst+sel for syst in SYSUNC ]
        #if isMC and addGenInfo: tmpList = tmpList + [ 'gen'+var+sel ] + [ 'resp'+var+syst+sel for syst in SYSUNC ]
        if isMC and addGenInfo: tmpList = tmpList + [ 'gen'+var+sel, 'missgen'+var+sel, 'accepgen'+var+sel, 'truereco'+var+'_nom'+sel, 'fakereco'+var+'_nom'+sel ] + [ 'resp'+var+syst+sel for syst in SYSUNC ]
        if respOnly: tmpList = [ 'resp'+var+syst+sel for syst in SYSUNC ] #+ [ 'missgen'+var+sel ]
        
        
        for ih in tmpList:
            print ('Processing '+isam+' '+ih)
            if isMC:
                allHistos[isam+'_'+ih] = samples[isam][0].Get( 'jetObservables/'+ih )
                tmpIsam = 'TT' if isam.startswith('data') else isam
                MCScale = samples[isam][1]['XS'] * lumi / samples[isam][1][year][('nGenWeights' if process.startswith('data') else 'nGenWeights') ]
                allHistos[isam+'_'+ih].Scale( MCScale )
            else:
                if sel.startswith('_dijet') and process.startswith('data'):
                    tmpdataHistos = {}
                    for it in checkDict( 'JetHT', dictSamples )[year]['triggerList']:
                        tmpdataHistos[ it ] = samples[isam][0].Get( 'jetObservables/'+ih.replace( sel, '_'+it+sel ) )
                        tmpdataHistos[ it ].Scale( checkDict( 'JetHT', dictSamples )[year]['triggerList'][it] )
                    allHistos[ isam+'_'+ih ] = tmpdataHistos[next(iter(tmpdataHistos))].Clone()
                    allHistos[ isam+'_'+ih ].Reset()
                    for i in tmpdataHistos: allHistos[isam+'_'+ih].Add( tmpdataHistos[i] )
                else:
                    allHistos[isam+'_'+ih] = samples[isam][0].Get( 'jetObservables/'+ih )
            #print (variables, var, isam, ih)
            if len(variables[var]['bins'])==1:
                genBin = variables[var]['bins'][0]
                recoBin = variables[var]['bins'][0]/2
            else:
                genBin = variables[var]['bins']
                recoBin = np.sort(np.append( variables[var]['bins'], np.array([ (variables[var]['bins'][i]+variables[var]['bins'][i+1])/2 for i in range(len(variables[var]['bins'])-1) ]  ) ))

            if not ih.startswith('resp'):
                if len(variables[var]['bins'])==1:
                    if ih.startswith(('reco','fake', 'true')):
                        allHistos[isam+'_'+ih+'_genBin'] = allHistos[isam+'_'+ih].Clone()
                        allHistos[isam+'_'+ih+'_genBin'].Rebin( genBin )
                        allHistos[isam+'_'+ih].Rebin( recoBin )
                    else: allHistos[isam+'_'+ih].Rebin( genBin )
                else:
                    if ih.startswith(('reco','fake','true')):
                        allHistos[isam+'_'+ih+'_genBin'] = allHistos[isam+'_'+ih].Clone()
                        allHistos[isam+'_'+ih+'_genBin'] = allHistos[isam+'_'+ih+'_genBin'].Rebin( len(genBin)-1, allHistos[isam+'_'+ih].GetName()+"_Rebin_genBin", array( 'd', genBin ) )
                        allHistos[isam+'_'+ih] = allHistos[isam+'_'+ih].Rebin( len(recoBin)-1, allHistos[isam+'_'+ih].GetName()+"_Rebin", array( 'd', recoBin ) )
                    else:
                        allHistos[isam+'_'+ih] = allHistos[isam+'_'+ih].Rebin( len(genBin)-1, allHistos[isam+'_'+ih].GetName()+"_Rebin", array( 'd', genBin ) )
            else:
                if len(variables[var]['bins'])==1: allHistos[isam+'_'+ih].Rebin2D( genBin, recoBin )
                else:

                    #### fancy way to create variable binning TH2D
                    tmpHisto = ROOT.TH2F( allHistos[isam+'_'+ih].GetName()+isam+"_Rebin", allHistos[isam+'_'+ih].GetName()+isam+"_Rebin", len(genBin)-1, array( 'd', genBin), len(recoBin)-1, array( 'd', recoBin) )

                    tmpArrayContent = np.zeros((len(genBin), len(recoBin)))
                    tmpArrayError = np.zeros((len(genBin), len(recoBin)))

#                    for biny in range( 1, allHistos[isam+'_'+ih].GetNbinsY()+1 ):
#                        for binx in range( 1, allHistos[isam+'_'+ih].GetNbinsX()+1 ):
#                            tmpHisto.Fill( allHistos[isam+'_'+ih].GetXaxis().GetBinCenter(binx), allHistos[isam+'_'+ih].GetYaxis().GetBinCenter(biny), allHistos[isam+'_'+ih].GetBinContent( binx, biny ) )

                    for biny in range( 1, allHistos[isam+'_'+ih].GetNbinsY()+1 ):
                        by = allHistos[isam+'_'+ih].GetYaxis().GetBinCenter( biny )
                        for binx in range( 1, allHistos[isam+'_'+ih].GetNbinsX()+1 ):
                            bx = allHistos[isam+'_'+ih].GetXaxis().GetBinCenter(binx)
                            for iX in range( len(genBin)-1 ):
                                for iY in range( len(recoBin)-1 ):
                                    if (bx<genBin[iX+1] and bx>genBin[iX]) and (by<recoBin[iY+1] and by>recoBin[iY]):
                                        jbin = allHistos[isam+'_'+ih].GetBin(binx,biny)
                                        tmpArrayContent[iX][iY] = tmpArrayContent[iX][iY] + allHistos[isam+'_'+ih].GetBinContent( jbin )
                                        tmpArrayError[iX][iY] = tmpArrayError[iX][iY] + ROOT.TMath.Power( allHistos[isam+'_'+ih].GetBinError( jbin ), 2 )

                    for biny in range( 1, tmpHisto.GetNbinsY()+1 ):
                        for binx in range( 1, tmpHisto.GetNbinsX()+1 ):
                            #if (binx <= ((biny+1)/2.)+4) and (binx >= ((biny+1)/2.)-4):
                            tmpHisto.SetBinContent( tmpHisto.GetBin(binx,biny), tmpArrayContent[binx-1][biny-1] )
                            tmpHisto.SetBinError( tmpHisto.GetBin(binx,biny), ROOT.TMath.Sqrt(tmpArrayError[binx-1][biny-1] ) )

                    tmpHisto.Sumw2()
                    allHistos[isam+'_'+ih] = tmpHisto

                #if isMC: allHistos[isam+'_'+ih].Scale( MCScale )
                ##### For tests, projections directly from 2D
                allHistos[isam+'_genJetfrom_'+ih] = allHistos[isam+'_'+ih].ProjectionY()
                allHistos[isam+'_recoJetfrom_'+ih] = allHistos[isam+'_'+ih].ProjectionX()


    tmpHistos = { k:v for (k,v) in allHistos.items() if 'Inf' in k }
    for ih in tmpHistos:
        for jh in allHistos:
            if (jh.endswith('0'+ih.split('Inf')[1])) and not ('Inf' in jh ):
                tmpHistos[ih].Add( allHistos[jh].Clone() )
    if len(tmpHistos)>0: allHistos = tmpHistos
    
    '''
    if len(samples)<2:
        
        #print (samples[isam][1][year]['skimmerHisto'])
        outFileName = samples[isam][1][year]['skimmerHisto'].split('.root')[0]+'_loadedHistos.root'
        print ("Written loaded/rebinned histos to:", outFileName)
        outFile = ROOT.TFile.Open ( '../test/Samples/'+outFileName ," RECREATE ")
        outHistFile.cd()
        for h in allHistos:
            h.Write()
        outFile.Close()
    '''
    return allHistos

In [13]:
def Fill2D(h0,h,binx,biny):
    Nx=h0.GetNbinsX()
    Ny=h0.GetNbinsY()
    hx=h0.ProjectionX()
    hy=h0.ProjectionY()
    content=np.zeros((len(binx)-1,len(biny)-1),dtype=float)
    for i in range(len(binx)-1):
        for j in range(len(biny)-1):
            content[i][j]=0
    for i in range(Nx):
        for j in range(Ny):
            centerx=hx.GetBinCenter(i+1)
            indexx=np.digitize(np.array([centerx]),binx)
            centery=hy.GetBinCenter(j+1)
            indexy=np.digitize(np.array([centery]),biny)
            if indexx[0] < len(binx) and indexy[0] < len(biny):
                content[indexx[0]-1][indexy[0]-1]+=h0.GetBinContent(i+1,j+1)
    for i in range(len(binx)-1):
        for j in range(len(biny)-1):
            h.SetBinContent(i+1,j+1,content[i][j])
            
def purity_stability(h0,bins):
    
    bins=np.array(bins)
    h=ROOT.TH2D("",";;",bins.shape[0]-1,(bins),bins.shape[0]-1,(bins))
    Fill2D(h0,h,bins,bins)
    hx=h.ProjectionX()
    hy=h.ProjectionY()
    N=hx.GetNbinsX()
    
    nreco_pergenbin=[]
    nreco_genanywhere=[]
    ngen_recoanywhere=[]
    for i in range(N):
        nreco_pergenbin.append(h.GetBinContent(i+1,i+1))
        nreco_genanywhere.append(hy.GetBinContent(i+1))
        ngen_recoanywhere.append(hx.GetBinContent(i+1))
    
    nreco_pergenbin=np.array(nreco_pergenbin)
    nreco_genanywhere=np.array(nreco_genanywhere)
    ngen_recoanywhere=np.array(ngen_recoanywhere)
    
    purity_arr=1.0*nreco_pergenbin/nreco_genanywhere
    stability_arr=1.0*nreco_pergenbin/ngen_recoanywhere
    
    for i in range(0,purity_arr.shape[0]):
        if np.isinf(purity_arr[i])==True or np.isnan(purity_arr[i])==True: purity_arr[i]=0.
        if np.isinf(stability_arr[i])==True or np.isnan(stability_arr[i])==True: stability_arr[i]=0.
            
    purity=ROOT.TH1D("Purity",";;",bins.shape[0]-1,(bins))
    stability=ROOT.TH1D("Stability",";;",bins.shape[0]-1,(bins))
    
    for i in range(N):
        purity.SetBinContent(i+1,purity_arr[i])
        stability.SetBinContent(i+1,stability_arr[i])
    
    purity.SetLineWidth(2)
    #purity.SetLineStyle(1)
    stability.SetLineWidth(2)
    #stability.SetLineStyle(1)
    
    return purity,stability

In [14]:
def runPurity( name, inputFiles, outputFolder, variables, sel, selection, year, version, ext, lumi, process='MC', only=False  ):

    numBins = 0
    numBinsList = [0]
    dictHistos = OrderedDict()
    for ivar in variables:
        #print (ivar.split('_')[0])
        if not name.endswith( ivar.split('_')[0] ): continue
        ### Getting input histos
        signalHistos = loadHistograms( inputFiles, ivar, sel, sysUnc=[], lumi=lumi, year=year, process=process, variables=variables )
        outputDir=outputFolder
        if not os.path.exists(outputDir): os.makedirs(outputDir)

        legend=ROOT.TLegend(0.15,0.80,0.90,0.90)
        legend.SetFillStyle(0)
        legend.SetBorderSize(0)
        legend.SetLineColor(0)

        legend.SetTextSize(0.04)
        legend.SetNColumns( 4 )
        #print (signalHistos)
        dictHistos[ 'acceptanceGraph_'+ivar ] = signalHistos[signalLabel+'_accepgen'+ivar+selection].Clone()
        dictHistos[ 'acceptanceGraph_'+ivar ].SetLineWidth(2)
        dictHistos[ 'acceptanceGraph_'+ivar ].SetLineColor(colorPallete[1])
        dictHistos[ 'acceptanceGraph_'+ivar ].SetLineStyle(2)
        dictHistos[ 'acceptanceGraph_'+ivar ].Divide(  signalHistos[signalLabel+'_gen'+ivar+selection] )
        legend.AddEntry( dictHistos[ 'acceptanceGraph_'+ivar ], 'Acceptance', 'l' )

        dictHistos[ 'fakeGraph_'+ivar ] = signalHistos[signalLabel+'_fakereco'+ivar+'_nom'+selection+'_genBin'].Clone()
        dictHistos[ 'fakeGraph_'+ivar ].SetLineWidth(2)
        dictHistos[ 'fakeGraph_'+ivar ].SetLineColor(colorPallete[2])
        dictHistos[ 'fakeGraph_'+ivar ].SetLineStyle(2)
        dictHistos[ 'fakeGraph_'+ivar ].Divide(  signalHistos[signalLabel+'_reco'+ivar+'_nom'+selection+'_genBin'] )
        legend.AddEntry( dictHistos[ 'fakeGraph_'+ivar ], 'Fakerate', 'l' )

        #### create diagonal->nope, 
        #### just do things the 'old-fashioned way' from the individual 1-D histos from the resp projection
        tmpHisto = signalHistos[signalLabel+'_resp'+ivar+'_nom'+sel].Clone()
        tmpHisto.Rebin2D( 1, 2 )
        p,s = purity_stability(tmpHisto,variables[ivar]['bins'])
        dictHistos[ 'purityGraph_'+ivar ] = p.Clone()
        #dictHistos[ 'purityGraph_'+ivar ].Reset()
        dictHistos[ 'stabilityGraph_'+ivar ] = s.Clone()
        
        #dictHistos[ 'purityGraph_'+ivar ].SetLineWidth(2)
        dictHistos[ 'purityGraph_'+ivar ].SetLineColor(colorPallete[3])
        #dictHistos[ 'purityGraph_'+ivar ].Divide(  signalHistos[signalLabel+'_reco'+ivar+'_nom'+selection+'_genBin'] )
        legend.AddEntry( dictHistos[ 'purityGraph_'+ivar ], 'Purity', 'l' )

        #dictHistos[ 'stabilityGraph_'+ivar ].SetLineWidth(2)
        dictHistos[ 'stabilityGraph_'+ivar ].SetLineColor(colorPallete[4])
        #dictHistos[ 'stabilityGraph_'+ivar ].Divide(  signalHistos[signalLabel+'_gen'+ivar+selection] )
        legend.AddEntry( dictHistos[ 'stabilityGraph_'+ivar ], 'Stability', 'l' )

        ##### removing tau21 and tau32 from total plots
        if not ivar.endswith(('21', '32')):
            numBins = numBins + dictHistos[ 'purityGraph_'+ivar ].GetNbinsX()
            numBinsList.append(numBins)

        multigraph = ROOT.THStack()
        for i in dictHistos:
            if i.endswith(ivar): multigraph.Add( dictHistos[i]  )

        ###### Plot unfolding results
        ROOT.gStyle.SetPadRightMargin(0.05)
        canvas = ROOT.TCanvas('canvas', 'canvas', 750, 500)

        multigraph.Draw("hist nostack")
        multigraph.GetXaxis().SetTitle( variables[ivar]['label'] )
        multigraph.GetYaxis().SetTitle( 'Percentage' )
        multigraph.SetMaximum( 1.1 )
        multigraph.GetYaxis().SetTitleOffset(0.8)

        legend.Draw()
        CMS_lumi.extraText = "Simulation"
        CMS_lumi.lumi_13TeV = "13 TeV, "+ ( '2017+2018' if year.startswith('all') else year )
        CMS_lumi.relPosX = 0.11
        CMS_lumi.CMS_lumi(canvas, 4, 0)

        textBox=ROOT.TLatex()
        textBox.SetNDC()
        textBox.SetTextSize(0.04)
        textBox.SetTextFont(62) ### 62 is bold, 42 is normal
        #textBox.DrawLatex(0.65, 0.75, sel.split('Sel')[0].split('_')[1]+' Selection' )

        canvas.SaveAs(outputDir+ivar+'_'+signalLabel+sel+'_Purity_'+version+'.'+ext)
        if ext.startswith('pdf'):
            canvas.SaveAs(outputDir+ivar+'_'+signalLabel+sel+'_Purity_'+version+'.png')


        ########## Plot response matrix
        print ('|------> Cross check: plotting response matrix for signal')
        ROOT.gStyle.SetPadRightMargin(0.15)
        #ROOT.gStyle.SetPalette(ROOT.kGistEarth)
        #ROOT.TColor.InvertPalette()
        can2D = ROOT.TCanvas(ivar+'can2D', ivar+'can2D', 750, 500 )
        signalHistos[signalLabel+'_resp'+ivar+'_nom'+sel].GetXaxis().SetTitle('Accepted Gen '+variables[ivar]['label'])
        signalHistos[signalLabel+'_resp'+ivar+'_nom'+sel].GetYaxis().SetTitle('True Reco '+variables[ivar]['label'])
        signalHistos[signalLabel+'_resp'+ivar+'_nom'+sel].GetYaxis().SetTitleOffset( 0.8 )
        signalHistos[signalLabel+'_resp'+ivar+'_nom'+sel].Draw("colz")
        CMS_lumi.relPosX = 0.12
        CMS_lumi.CMS_lumi(can2D, 4, 0)
        can2D.SaveAs(outputDir+ivar+'_'+signalLabel+sel+'_responseMatrix'+version+'.'+ext)
        if ext.startswith('pdf'):
            can2D.SaveAs(outputDir+ivar+'_'+signalLabel+sel+'_responseMatrix'+version+'.png')

    if not only:

        outputFileName = name+'_purity_'+signalLabel+'_combinePlots_'+version+'.'+ext
        print('Processing.......', outputFileName)

        legend=ROOT.TLegend(0.18,0.82,0.58,0.92)
        legend.SetNColumns(2)
        legend.SetFillStyle(0)
        legend.SetTextSize(0.06)
        legend.SetBorderSize(0)
        
        dictHistos['purity'] = ROOT.TH1F('purity', 'purity', numBinsList[-1], 0, numBinsList[-1])
        legend.AddEntry( dictHistos[ 'purity' ], 'Purity', 'lep' )
        dictHistos['stability'] = ROOT.TH1F('stability', 'stability', numBinsList[-1], 0, numBinsList[-1])
        legend.AddEntry( dictHistos[ 'stability' ], 'Stability', 'lep' )

        tmpNbin = 0
        Xlabels = []
        for ivar in variables:
            if not name.endswith( ivar.split('_')[0] ): continue
            else:
                Xlabels.append( variables[ivar]['label'])
                for ibin in range(1, dictHistos['purityGraph_'+ivar].GetNbinsX()+1):
                    tmpNbin = tmpNbin+1
                    dictHistos['purity'].SetBinContent( tmpNbin, dictHistos['purityGraph_'+ivar].GetBinContent(ibin) )
                    dictHistos['stability'].SetBinContent( tmpNbin, dictHistos['stabilityGraph_'+ivar].GetBinContent(ibin) )

        ROOT.gStyle.SetPadRightMargin(0.05)
        #ROOT.gStyle.SetPadLeftMargin(0.08)
        canvasoutputFileName = ROOT.TCanvas('c1purity', 'c1purity', 1400, 500 )
        ROOT.gStyle.SetPadTickX(0)
        #pad1.SetLogy()
        dictHistos['purity'].GetXaxis().SetNdivisions(100)
        dictHistos['purity'].GetYaxis().SetTitle( '#frac{1}{d#sigma} #frac{d#sigma}{d#tau_{X}}' )
        dictHistos['purity'].GetYaxis().SetTitleSize( 0.06 )
        dictHistos['purity'].GetYaxis().SetTitleOffset( 0.8 )
        #dictHistos['purity'].SetMaximum( dictHistos['stability'].GetMaximum()*1.2 )
        dictHistos['purity'].SetMaximum( 1. )
        dictHistos['purity'].SetMinimum( 0.2 )
        dictHistos['purity'].SetLineColor( ROOT.kBlack )
        dictHistos['purity'].SetMarkerColor( ROOT.kBlack )
        dictHistos['purity'].SetMarkerSize( 0.5 )
        dictHistos['purity'].SetLineWidth( 2 )

        dictHistos['stability'].SetLineColor( ROOT.kMagenta )
        dictHistos['stability'].SetMarkerColor( ROOT.kMagenta )
        dictHistos['stability'].SetMarkerSize( 0.5 )
        dictHistos['stability'].SetLineWidth( 2 )
        dictHistos['purity'].Draw('hist')
        dictHistos[ 'stability' ].Draw('hist same')

        ### division lines
        lines = {}
        for i in numBinsList[1:-1]:
            lines[i] = ROOT.TGraph(2, array('d', [i,i]), array('d', [0, 200]) )
            lines[i].SetLineColor(ROOT.kGray)
            lines[i].Draw('same')

        CMS_lumi.lumiTextSize = 0.6
        CMS_lumi.relPosX = 0.07
        CMS_lumi.CMS_lumi( canvasoutputFileName, 4, 0)
        legend.Draw()

        textBox=ROOT.TLatex()
        #textBox.SetNDC()
        textBox.SetTextSize(0.06)
        textBox.SetTextAlign(12)
        textBoxList = {}
        for i in range(1, len(numBinsList)):
            textBoxList[i] = textBox.Clone()
            textBoxList[i].DrawLatex(numBinsList[i-1]+(numBinsList[i]-numBinsList[i-1])/2., 
                                     0.15, Xlabels[i-1] )
            print (Xlabels[i-1] )
        outputDir=outputFolder
        canvasoutputFileName.SaveAs( outputDir+'/'+outputFileName )
        if ext.startswith('pdf'):
            canvasoutputFileName.SaveAs( outputDir+'/'+outputFileName.replace('pdf', 'png') )
        ROOT.gStyle.SetPadTickX(1)

###  W selection

In [15]:
from variables_WSel import nSubVariables_WSel
from variables_topSel import nSubVariables_topSel


####  UL17

In [16]:
signalLabelBegin = 'TTToSemiLeptonic'

signalLabel = 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8'
inputFolder = '../test/Samples/'

sigFiles = {}

for iy in ( ['2017']):
    for isam in dictSamples:
        if not checkDict( isam, dictSamples )[iy]['skimmerHisto'].endswith('root'): continue
        if isam.startswith(signalLabelBegin):
            sigFiles[isam] = [
                            ROOT.TFile.Open(inputFolder+checkDict( isam, dictSamples )[iy]['skimmerHisto'] ),
                            checkDict( isam, dictSamples )
                        ]
print (sigFiles)
variables=nSubVariables_WSel
runPurity(name='recoJet', outputFolder='../test/Results_Local/WSel/UL17/BinningTests/', 
          inputFiles=sigFiles, variables=nSubVariables_WSel, sel='_WSel', 
          selection='_WSel', year='2017', version='_October', ext='png', lumi=1., process='MC')
runPurity(name='recoJet', outputFolder='../test/Results_Local/WSel/UL17/BinningTests/', 
          inputFiles=sigFiles, variables=nSubVariables_WSel, sel='_WSel', 
          selection='_WSel', year='2017', version='_October', ext='pdf', lumi=1., process='MC')

{'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8': [<cppyy.gbl.TFile object at 0x561102731060>, {'2017': {'skimmerHisto': 'TTToSemileptonic_powheg_pythia8_2017UL_nom.root', 'nGenWeights': 32392787477.907993}, '2018': {'skimmerHisto': 'TTToSemileptonic_powheg_pythia8_2018UL_nom.root', 'nGenWeights': 31378075292.893986}, 'all': {'skimmerHisto': 'TTToSemileptonic_powheg_pythia8_nom_UL17-18.root', 'nGenWeights': 63770862770.80198}, 'XS': 365.34, 'selection': 'Wtop', 'label': 't#bar{t} powheg', 'color': 616}]}
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_0p5_1_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_0p5_1_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_0p5_1_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 accepgenJet_tau_0p5_1_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 truerecoJet_tau_0p5_1_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 fakerecoJet_tau

/home/kaustuv1993/miniconda3/envs/HEPML/lib/python3.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in true_divide


|------> Cross check: plotting response matrix for signal
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_0p5_2_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_0p5_2_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_0p5_2_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 accepgenJet_tau_0p5_2_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 truerecoJet_tau_0p5_2_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 fakerecoJet_tau_0p5_2_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 respJet_tau_0p5_2_nom_WSel
|------> Cross check: plotting response matrix for signal
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_1_2_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_1_2_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_1_2_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythi

/home/kaustuv1993/miniconda3/envs/HEPML/lib/python3.7/site-packages/ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in true_divide


|------> Cross check: plotting response matrix for signal
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_1_3_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_1_3_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_1_3_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 accepgenJet_tau_1_3_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 truerecoJet_tau_1_3_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 fakerecoJet_tau_1_3_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 respJet_tau_1_3_nom_WSel
|------> Cross check: plotting response matrix for signal
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_2_3_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_2_3_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_2_3_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 accepgenJet

|------> Cross check: plotting response matrix for signal
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_0p5_3_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_0p5_3_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_0p5_3_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 accepgenJet_tau_0p5_3_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 truerecoJet_tau_0p5_3_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 fakerecoJet_tau_0p5_3_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 respJet_tau_0p5_3_nom_WSel
|------> Cross check: plotting response matrix for signal
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_1_3_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_1_3_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_1_3_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythi

Warning in <TH2F::Sumw2>: Sum of squares of weights structure already created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: png file ../test/Results_Local/WSel/UL17/BinningTests/Jet_tau_0p5_1_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_WSel_Purity__October.png has been created
Info in <TCanvas::Print>: png file ../test/Results_Local/WSel/UL17/BinningTests/Jet_tau_0p5_1_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_WSel_responseMatrix_October.png has been created
Warning in <TH2F::Sumw2>: Sum of squares of weights structure already created
Warning in <TFile::Append>: Replacing existing TH1: Purity (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: Purity (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: Stability (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: Stability (Potential memory leak).
Warning in <TCanvas::Constructor>: Deleting canvas with s

####  UL18

In [17]:
signalLabelBegin = 'TTToSemiLeptonic'

signalLabel = 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8'
inputFolder = '../test/Samples/'

sigFiles = {}

for iy in ( ['2018']):
    for isam in dictSamples:
        if not checkDict( isam, dictSamples )[iy]['skimmerHisto'].endswith('root'): continue
        if isam.startswith(signalLabelBegin):
            sigFiles[isam] = [
                            ROOT.TFile.Open(inputFolder+checkDict( isam, dictSamples )[iy]['skimmerHisto'] ),
                            checkDict( isam, dictSamples )
                        ]
print (sigFiles)
variables=nSubVariables_WSel
runPurity(name='recoJet', outputFolder='../test/Results_Local/WSel/UL18/BinningTests/', 
          inputFiles=sigFiles, variables=nSubVariables_WSel, sel='_WSel', 
          selection='_WSel', year='2018', version='_October', ext='png', lumi=1., process='MC')
runPurity(name='recoJet', outputFolder='../test/Results_Local/WSel/UL18/BinningTests/', 
          inputFiles=sigFiles, variables=nSubVariables_WSel, sel='_WSel', 
          selection='_WSel', year='2018', version='_October', ext='pdf', lumi=1., process='MC')

{'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8': [<cppyy.gbl.TFile object at 0x561105d2d070>, {'2017': {'skimmerHisto': 'TTToSemileptonic_powheg_pythia8_2017UL_nom.root', 'nGenWeights': 32392787477.907993}, '2018': {'skimmerHisto': 'TTToSemileptonic_powheg_pythia8_2018UL_nom.root', 'nGenWeights': 31378075292.893986}, 'all': {'skimmerHisto': 'TTToSemileptonic_powheg_pythia8_nom_UL17-18.root', 'nGenWeights': 63770862770.80198}, 'XS': 365.34, 'selection': 'Wtop', 'label': 't#bar{t} powheg', 'color': 616}]}
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_0p5_1_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_0p5_1_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_0p5_1_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 accepgenJet_tau_0p5_1_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 truerecoJet_tau_0p5_1_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 fakerecoJet_tau

/home/kaustuv1993/miniconda3/envs/HEPML/lib/python3.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in true_divide


|------> Cross check: plotting response matrix for signal
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_0p5_2_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_0p5_2_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_0p5_2_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 accepgenJet_tau_0p5_2_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 truerecoJet_tau_0p5_2_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 fakerecoJet_tau_0p5_2_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 respJet_tau_0p5_2_nom_WSel
|------> Cross check: plotting response matrix for signal
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_1_2_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_1_2_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_1_2_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythi

/home/kaustuv1993/miniconda3/envs/HEPML/lib/python3.7/site-packages/ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in true_divide


|------> Cross check: plotting response matrix for signal
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_1_3_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_1_3_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_1_3_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 accepgenJet_tau_1_3_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 truerecoJet_tau_1_3_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 fakerecoJet_tau_1_3_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 respJet_tau_1_3_nom_WSel
|------> Cross check: plotting response matrix for signal
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_2_3_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_2_3_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_2_3_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 accepgenJet

|------> Cross check: plotting response matrix for signal
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_0p5_3_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_0p5_3_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_0p5_3_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 accepgenJet_tau_0p5_3_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 truerecoJet_tau_0p5_3_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 fakerecoJet_tau_0p5_3_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 respJet_tau_0p5_3_nom_WSel
|------> Cross check: plotting response matrix for signal
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_1_3_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_1_3_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_1_3_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythi

Warning in <TH2F::Sumw2>: Sum of squares of weights structure already created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: png file ../test/Results_Local/WSel/UL18/BinningTests/Jet_tau_0p5_1_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_WSel_Purity__October.png has been created
Info in <TCanvas::Print>: png file ../test/Results_Local/WSel/UL18/BinningTests/Jet_tau_0p5_1_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_WSel_responseMatrix_October.png has been created
Warning in <TH2F::Sumw2>: Sum of squares of weights structure already created
Warning in <TFile::Append>: Replacing existing TH1: Purity (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: Purity (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: Stability (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: Stability (Potential memory leak).
Warning in <TCanvas::Constructor>: Deleting canvas with s

####  UL17-18

In [22]:
signalLabelBegin = 'TTToSemiLeptonic'

signalLabel = 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8'
inputFolder = '../test/Samples/'

sigFiles = {}

for iy in ( ['all']):
    for isam in dictSamples:
        print (checkDict( isam, dictSamples ))
        if not checkDict( isam, dictSamples )[iy]['skimmerHisto'].endswith('root'): continue
        if isam.startswith(signalLabelBegin):
            sigFiles[isam] = [
                            ROOT.TFile.Open(inputFolder+checkDict( isam, dictSamples )[iy]['skimmerHisto'] ),
                            checkDict( isam, dictSamples )
                        ]
print (sigFiles)
variables=nSubVariables_WSel
runPurity(name='recoJet', outputFolder='../test/Results_Local/WSel/UL17-18/BinningTests/', 
          inputFiles=sigFiles, variables=nSubVariables_WSel, sel='_WSel', 
          selection='_WSel', year='all', version='_October', ext='png', lumi=1., process='MC')
runPurity(name='recoJet', outputFolder='../test/Results_Local/WSel/UL17-18/BinningTests/', 
          inputFiles=sigFiles, variables=nSubVariables_WSel, sel='_WSel', 
          selection='_WSel', year='all', version='_October', ext='pdf', lumi=1., process='MC')

{'2017': {'skimmerHisto': 'SingleMuonAll_2017UL.root', 'lumi': 41472.052}, '2018': {'skimmerHisto': 'SingleMuonAll_2018UL.root', 'lumi': 59816.0}, 'all': {'skimmerHisto': 'SingleMuonAll_UL17-18.root', 'lumi': 59816.0}, 'color': 0, 'selection': 'Wtop'}
{'2017': {'skimmerHisto': 'TTJets_amcatnloFXFX-pythia8_2017UL.root', 'nGenWeights': 286629789811.7677}, '2018': {'skimmerHisto': 'TTJets_amcatnloFXFX-pythia8_2018UL.root', 'nGenWeights': 201969595228.20978}, 'all': {'skimmerHisto': 'TTJets_amcatnloFXFX-pythia8_UL17-18.root', 'nGenWeights': 488599385039.9775}, 'XS': 759.1, 'selection': 'Wtop', 'label': 't#bar{t} amcatnloFXFX', 'color': 600}
{'2017': {'skimmerHisto': 'TTToSemileptonic_powheg_pythia8_2017UL_nom.root', 'nGenWeights': 32392787477.907993}, '2018': {'skimmerHisto': 'TTToSemileptonic_powheg_pythia8_2018UL_nom.root', 'nGenWeights': 31378075292.893986}, 'all': {'skimmerHisto': 'TTToSemileptonic_powheg_pythia8_nom_UL17-18.root', 'nGenWeights': 63770862770.80198}, 'XS': 365.34, 'sele

|------> Cross check: plotting response matrix for signal
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_1_1_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_1_1_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_1_1_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 accepgenJet_tau_1_1_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 truerecoJet_tau_1_1_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 fakerecoJet_tau_1_1_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 respJet_tau_1_1_nom_WSel
|------> Cross check: plotting response matrix for signal
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_2_1_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_2_1_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_2_1_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 accepgenJet

/home/kaustuv1993/miniconda3/envs/HEPML/lib/python3.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in true_divide


|------> Cross check: plotting response matrix for signal
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_0p5_2_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_0p5_2_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_0p5_2_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 accepgenJet_tau_0p5_2_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 truerecoJet_tau_0p5_2_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 fakerecoJet_tau_0p5_2_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 respJet_tau_0p5_2_nom_WSel
|------> Cross check: plotting response matrix for signal
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_1_2_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_1_2_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_1_2_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythi

/home/kaustuv1993/miniconda3/envs/HEPML/lib/python3.7/site-packages/ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in true_divide


|------> Cross check: plotting response matrix for signal
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_1_3_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_1_3_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_1_3_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 accepgenJet_tau_1_3_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 truerecoJet_tau_1_3_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 fakerecoJet_tau_1_3_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 respJet_tau_1_3_nom_WSel
|------> Cross check: plotting response matrix for signal
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_2_3_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_2_3_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_2_3_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 accepgenJet

|------> Cross check: plotting response matrix for signal
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_0p5_3_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_0p5_3_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_0p5_3_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 accepgenJet_tau_0p5_3_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 truerecoJet_tau_0p5_3_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 fakerecoJet_tau_0p5_3_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 respJet_tau_0p5_3_nom_WSel
|------> Cross check: plotting response matrix for signal
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_1_3_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_1_3_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_1_3_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythi

Warning in <TH2F::Sumw2>: Sum of squares of weights structure already created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: png file ../test/Results_Local/WSel/UL17-18/BinningTests/Jet_tau_0p5_1_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_WSel_Purity__October.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: Jet_tau_0p5_1can2D
Info in <TCanvas::Print>: png file ../test/Results_Local/WSel/UL17-18/BinningTests/Jet_tau_0p5_1_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_WSel_responseMatrix_October.png has been created
Warning in <TH2F::Sumw2>: Sum of squares of weights structure already created
Warning in <TFile::Append>: Replacing existing TH1: Purity (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: Purity (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: Stability (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: 

###  top selection

####  UL17

In [18]:
signalLabelBegin = 'TTToSemiLeptonic'

signalLabel = 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8'
inputFolder = '../test/Samples/'

sigFiles = {}

for iy in ( ['2017']):
    for isam in dictSamples:
        if not checkDict( isam, dictSamples )[iy]['skimmerHisto'].endswith('root'): continue
        if isam.startswith(signalLabelBegin):
            sigFiles[isam] = [
                            ROOT.TFile.Open(inputFolder+checkDict( isam, dictSamples )[iy]['skimmerHisto'] ),
                            checkDict( isam, dictSamples )
                        ]
print (sigFiles)
variables=nSubVariables_topSel
runPurity(name='recoJet', outputFolder='../test/Results_Local/topSel/UL17/BinningTests/', 
          inputFiles=sigFiles, variables=nSubVariables_topSel, sel='_topSel', 
          selection='_topSel', year='2017', version='_October', ext='png', lumi=1., process='MC')
runPurity(name='recoJet', outputFolder='../test/Results_Local/topSel/UL17/BinningTests/', 
          inputFiles=sigFiles, variables=nSubVariables_topSel, sel='_topSel', 
          selection='_topSel', year='2017', version='_October', ext='pdf', lumi=1., process='MC')

{'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8': [<cppyy.gbl.TFile object at 0x561105d2d070>, {'2017': {'skimmerHisto': 'TTToSemileptonic_powheg_pythia8_2017UL_nom.root', 'nGenWeights': 32392787477.907993}, '2018': {'skimmerHisto': 'TTToSemileptonic_powheg_pythia8_2018UL_nom.root', 'nGenWeights': 31378075292.893986}, 'all': {'skimmerHisto': 'TTToSemileptonic_powheg_pythia8_nom_UL17-18.root', 'nGenWeights': 63770862770.80198}, 'XS': 365.34, 'selection': 'Wtop', 'label': 't#bar{t} powheg', 'color': 616}]}
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_0p5_1_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_0p5_1_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_0p5_1_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 accepgenJet_tau_0p5_1_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 truerecoJet_tau_0p5_1_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 faker

/home/kaustuv1993/miniconda3/envs/HEPML/lib/python3.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in true_divide
/home/kaustuv1993/miniconda3/envs/HEPML/lib/python3.7/site-packages/ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in true_divide


|------> Cross check: plotting response matrix for signal
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_1_1_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_1_1_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_1_1_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 accepgenJet_tau_1_1_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 truerecoJet_tau_1_1_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 fakerecoJet_tau_1_1_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 respJet_tau_1_1_nom_topSel
|------> Cross check: plotting response matrix for signal
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_2_1_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_2_1_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_2_1_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg

|------> Cross check: plotting response matrix for signal
Processing....... recoJet_purity_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_combinePlots__October.png
#tau_{1}^{(0.5)}
#tau_{1}^{(1)}
#tau_{1}^{(2)}
#tau_{2}^{(0.5)}
#tau_{2}^{(1)}
#tau_{2}^{(2)}
#tau_{3}^{(0.5)}
#tau_{3}^{(1)}
#tau_{3}^{(2)}
#tau_{4}^{(0.5)}
#tau_{4}^{(1)}
#tau_{4}^{(2)}
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_0p5_1_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_0p5_1_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_0p5_1_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 accepgenJet_tau_0p5_1_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 truerecoJet_tau_0p5_1_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 fakerecoJet_tau_0p5_1_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 respJet_tau_0p5_1_nom_topSel
|------> Cross check: plotting response matri

|------> Cross check: plotting response matrix for signal
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau32_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau32_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau32_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 accepgenJet_tau32_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 truerecoJet_tau32_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 fakerecoJet_tau32_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 respJet_tau32_nom_topSel
|------> Cross check: plotting response matrix for signal
Processing....... recoJet_purity_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_combinePlots__October.pdf
#tau_{1}^{(0.5)}
#tau_{1}^{(1)}
#tau_{1}^{(2)}
#tau_{2}^{(0.5)}
#tau_{2}^{(1)}
#tau_{2}^{(2)}
#tau_{3}^{(0.5)}
#tau_{3}^{(1)}
#tau_{3}^{(2)}
#tau_{4}^{(0.5)}
#tau_{4}^{(1)}
#tau_{4}^{(2)}


Warning in <TH2F::Sumw2>: Sum of squares of weights structure already created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: png file ../test/Results_Local/topSel/UL17/BinningTests/Jet_tau_0p5_1_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_topSel_Purity__October.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: Jet_tau_0p5_1can2D
Info in <TCanvas::Print>: png file ../test/Results_Local/topSel/UL17/BinningTests/Jet_tau_0p5_1_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_topSel_responseMatrix_October.png has been created
Warning in <TH2F::Sumw2>: Sum of squares of weights structure already created
Warning in <TFile::Append>: Replacing existing TH1: Purity (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: Purity (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: Stability (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1

####  UL18

In [19]:
signalLabelBegin = 'TTToSemiLeptonic'

signalLabel = 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8'
inputFolder = '../test/Samples/'

sigFiles = {}

for iy in ( ['2018']):
    for isam in dictSamples:
        if not checkDict( isam, dictSamples )[iy]['skimmerHisto'].endswith('root'): continue
        if isam.startswith(signalLabelBegin):
            sigFiles[isam] = [
                            ROOT.TFile.Open(inputFolder+checkDict( isam, dictSamples )[iy]['skimmerHisto'] ),
                            checkDict( isam, dictSamples )
                        ]
print (sigFiles)
variables=nSubVariables_topSel
runPurity(name='recoJet', outputFolder='../test/Results_Local/topSel/UL18/BinningTests/', 
          inputFiles=sigFiles, variables=nSubVariables_topSel, sel='_topSel', 
          selection='_topSel', year='2018', version='_October', ext='png', lumi=1., process='MC')
runPurity(name='recoJet', outputFolder='../test/Results_Local/topSel/UL18/BinningTests/', 
          inputFiles=sigFiles, variables=nSubVariables_topSel, sel='_topSel', 
          selection='_topSel', year='2018', version='_October', ext='pdf', lumi=1., process='MC')

{'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8': [<cppyy.gbl.TFile object at 0x561105d2d070>, {'2017': {'skimmerHisto': 'TTToSemileptonic_powheg_pythia8_2017UL_nom.root', 'nGenWeights': 32392787477.907993}, '2018': {'skimmerHisto': 'TTToSemileptonic_powheg_pythia8_2018UL_nom.root', 'nGenWeights': 31378075292.893986}, 'all': {'skimmerHisto': 'TTToSemileptonic_powheg_pythia8_nom_UL17-18.root', 'nGenWeights': 63770862770.80198}, 'XS': 365.34, 'selection': 'Wtop', 'label': 't#bar{t} powheg', 'color': 616}]}
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_0p5_1_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_0p5_1_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_0p5_1_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 accepgenJet_tau_0p5_1_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 truerecoJet_tau_0p5_1_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 faker

/home/kaustuv1993/miniconda3/envs/HEPML/lib/python3.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in true_divide
/home/kaustuv1993/miniconda3/envs/HEPML/lib/python3.7/site-packages/ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in true_divide


|------> Cross check: plotting response matrix for signal
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_1_1_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_1_1_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_1_1_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 accepgenJet_tau_1_1_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 truerecoJet_tau_1_1_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 fakerecoJet_tau_1_1_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 respJet_tau_1_1_nom_topSel
|------> Cross check: plotting response matrix for signal
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_2_1_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_2_1_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_2_1_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg

|------> Cross check: plotting response matrix for signal
Processing....... recoJet_purity_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_combinePlots__October.png
#tau_{1}^{(0.5)}
#tau_{1}^{(1)}
#tau_{1}^{(2)}
#tau_{2}^{(0.5)}
#tau_{2}^{(1)}
#tau_{2}^{(2)}
#tau_{3}^{(0.5)}
#tau_{3}^{(1)}
#tau_{3}^{(2)}
#tau_{4}^{(0.5)}
#tau_{4}^{(1)}
#tau_{4}^{(2)}
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_0p5_1_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_0p5_1_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_0p5_1_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 accepgenJet_tau_0p5_1_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 truerecoJet_tau_0p5_1_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 fakerecoJet_tau_0p5_1_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 respJet_tau_0p5_1_nom_topSel
|------> Cross check: plotting response matri

|------> Cross check: plotting response matrix for signal
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau32_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau32_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau32_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 accepgenJet_tau32_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 truerecoJet_tau32_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 fakerecoJet_tau32_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 respJet_tau32_nom_topSel
|------> Cross check: plotting response matrix for signal
Processing....... recoJet_purity_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_combinePlots__October.pdf
#tau_{1}^{(0.5)}
#tau_{1}^{(1)}
#tau_{1}^{(2)}
#tau_{2}^{(0.5)}
#tau_{2}^{(1)}
#tau_{2}^{(2)}
#tau_{3}^{(0.5)}
#tau_{3}^{(1)}
#tau_{3}^{(2)}
#tau_{4}^{(0.5)}
#tau_{4}^{(1)}
#tau_{4}^{(2)}


Warning in <TH2F::Sumw2>: Sum of squares of weights structure already created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: png file ../test/Results_Local/topSel/UL18/BinningTests/Jet_tau_0p5_1_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_topSel_Purity__October.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: Jet_tau_0p5_1can2D
Info in <TCanvas::Print>: png file ../test/Results_Local/topSel/UL18/BinningTests/Jet_tau_0p5_1_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_topSel_responseMatrix_October.png has been created
Warning in <TH2F::Sumw2>: Sum of squares of weights structure already created
Warning in <TFile::Append>: Replacing existing TH1: Purity (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: Purity (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: Stability (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1

####  UL17-18

In [23]:
signalLabelBegin = 'TTToSemiLeptonic'

signalLabel = 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8'
inputFolder = '../test/Samples/'

sigFiles = {}

for iy in ( ['all']):
    for isam in dictSamples:
        if not checkDict( isam, dictSamples )[iy]['skimmerHisto'].endswith('root'): continue
        if isam.startswith(signalLabelBegin):
            sigFiles[isam] = [
                            ROOT.TFile.Open(inputFolder+checkDict( isam, dictSamples )[iy]['skimmerHisto'] ),
                            checkDict( isam, dictSamples )
                        ]
print (sigFiles)
variables=nSubVariables_topSel
runPurity(name='recoJet', outputFolder='../test/Results_Local/topSel/UL17-18/BinningTests/', 
          inputFiles=sigFiles, variables=nSubVariables_topSel, sel='_topSel', 
          selection='_topSel', year='all', version='_October', ext='png', lumi=1., process='MC')
runPurity(name='recoJet', outputFolder='../test/Results_Local/topSel/UL17-18/BinningTests/', 
          inputFiles=sigFiles, variables=nSubVariables_topSel, sel='_topSel', 
          selection='_topSel', year='all', version='_October', ext='pdf', lumi=1., process='MC')

{'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8': [<cppyy.gbl.TFile object at 0x561104d66560>, {'2017': {'skimmerHisto': 'TTToSemileptonic_powheg_pythia8_2017UL_nom.root', 'nGenWeights': 32392787477.907993}, '2018': {'skimmerHisto': 'TTToSemileptonic_powheg_pythia8_2018UL_nom.root', 'nGenWeights': 31378075292.893986}, 'all': {'skimmerHisto': 'TTToSemileptonic_powheg_pythia8_nom_UL17-18.root', 'nGenWeights': 63770862770.80198}, 'XS': 365.34, 'selection': 'Wtop', 'label': 't#bar{t} powheg', 'color': 616}]}
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_0p5_1_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_0p5_1_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_0p5_1_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 accepgenJet_tau_0p5_1_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 truerecoJet_tau_0p5_1_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 faker

/home/kaustuv1993/miniconda3/envs/HEPML/lib/python3.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in true_divide
/home/kaustuv1993/miniconda3/envs/HEPML/lib/python3.7/site-packages/ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in true_divide


|------> Cross check: plotting response matrix for signal
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_1_1_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_1_1_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_1_1_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 accepgenJet_tau_1_1_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 truerecoJet_tau_1_1_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 fakerecoJet_tau_1_1_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 respJet_tau_1_1_nom_topSel
|------> Cross check: plotting response matrix for signal
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_2_1_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_2_1_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_2_1_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg

|------> Cross check: plotting response matrix for signal
Processing....... recoJet_purity_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_combinePlots__October.png
#tau_{1}^{(0.5)}
#tau_{1}^{(1)}
#tau_{1}^{(2)}
#tau_{2}^{(0.5)}
#tau_{2}^{(1)}
#tau_{2}^{(2)}
#tau_{3}^{(0.5)}
#tau_{3}^{(1)}
#tau_{3}^{(2)}
#tau_{4}^{(0.5)}
#tau_{4}^{(1)}
#tau_{4}^{(2)}
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_0p5_1_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_0p5_1_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_0p5_1_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 accepgenJet_tau_0p5_1_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 truerecoJet_tau_0p5_1_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 fakerecoJet_tau_0p5_1_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 respJet_tau_0p5_1_nom_topSel
|------> Cross check: plotting response matri

|------> Cross check: plotting response matrix for signal
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau32_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau32_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau32_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 accepgenJet_tau32_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 truerecoJet_tau32_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 fakerecoJet_tau32_nom_topSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 respJet_tau32_nom_topSel
|------> Cross check: plotting response matrix for signal
Processing....... recoJet_purity_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_combinePlots__October.pdf
#tau_{1}^{(0.5)}
#tau_{1}^{(1)}
#tau_{1}^{(2)}
#tau_{2}^{(0.5)}
#tau_{2}^{(1)}
#tau_{2}^{(2)}
#tau_{3}^{(0.5)}
#tau_{3}^{(1)}
#tau_{3}^{(2)}
#tau_{4}^{(0.5)}
#tau_{4}^{(1)}
#tau_{4}^{(2)}


Warning in <TH2F::Sumw2>: Sum of squares of weights structure already created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: png file ../test/Results_Local/topSel/UL17-18/BinningTests/Jet_tau_0p5_1_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_topSel_Purity__October.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: Jet_tau_0p5_1can2D
Info in <TCanvas::Print>: png file ../test/Results_Local/topSel/UL17-18/BinningTests/Jet_tau_0p5_1_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_topSel_responseMatrix_October.png has been created
Warning in <TH2F::Sumw2>: Sum of squares of weights structure already created
Warning in <TFile::Append>: Replacing existing TH1: Purity (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: Purity (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: Stability (Potential memory leak).
Warning in <TFile::Append>: Replacing existi